### Data Cleaning

- Here we are combining the data of Indeed and Seek into one data frame and perform all the clenaing process

In [1]:
import pandas as pd
import numpy as np
import re

import warnings
warnings.simplefilter(action='ignore')#/, category=FutureWarning)

In [1]:
indeed_file = './data/indeed_data.csv'
seek_file = './data/seek_data.csv'

In [3]:
df_indeed_data = pd.read_csv(indeed_file)
df_seek_data = pd.read_csv(seek_file)

In [4]:
df_indeed_data.shape ,  df_seek_data.shape

((337, 5), (1829, 5))

In [5]:
df_seek_data['salary'].fillna('None',inplace=True)
df_seek_data['job_desc'].fillna('None',inplace=True)

In [6]:
df_job_data = pd.concat([df_seek_data, df_indeed_data])

In [7]:
df_job_data.shape

(2166, 5)

In [8]:
df_job_data.reset_index(inplace=True,drop=True)

In [9]:
df_job_data.head()

,job_title,location,salary,company,job_desc
0,Data Analytics Consultant / BI Developer,Brisbane,None,Aginic,None
1,Data and Insights Reporting Analyst.,Sydney,None,ELMO Software Limited,"<div class=""tempmargin"">\r\n <h1 class=""j..."
2,Data Analyst,Sydney,Competitive Day Rate,Hydrogen Group Pty Ltd,"<div class=""tempmargin"">\r\n <h1 class=""j..."
3,Data Analyst,Sydney,"$75,000 - $84,999 base + super",Salient Group,"<div class=""tempmargin"">\r\n <h1 class=""j..."
4,Data Visualisation Analyst,Melbourne,Super,Ambition Technology,"<div class=""tempmargin"">\r\n <h1 class=""j..."


In [10]:
#create a function to remove tags and newline character from job descirption
def remove_tags(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext
def remove_newline_char(raw_html):
    cleanr = re.compile(r'[^ \w\.]')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [11]:
#removing html tags
df_job_data['job_desc'] = df_job_data['job_desc'].apply(lambda x: remove_tags(x) if x!='None' else x)

#removing new line character
df_job_data['job_desc'] = df_job_data['job_desc'].apply(lambda x: remove_newline_char(x) if x!='None' else x)

#removing punctuation mark
df_job_data['job_desc'] = df_job_data['job_desc'].str.replace('[^\w\s]',' ')

#removing double spaces if any
df_job_data['job_desc'] = df_job_data['job_desc'].str.replace('      ',' ')

#converting title and text into lower case
df_job_data['job_desc'] = df_job_data['job_desc'].apply(lambda x: x.lower() if x!='None' else x)


In [12]:
#salary cleaning

#removing new line character if any
df_job_data['salary'] = df_job_data['salary'].str.replace('\n', '')

#removing ',' from salary
df_job_data['salary'] = df_job_data['salary'].str.replace(',', '')

#removing '$' from salary
df_job_data['salary'] = df_job_data['salary'].str.replace('$', '')

#converting the slares into lower case
df_job_data['salary'] = df_job_data['salary'].apply(lambda x: x.lower())

In [13]:
df_job_data['salary_period'] = np.nan
#if the salary contains information on time period, save that time
#period string in the og_salary_period column
df_job_data.ix[df_job_data['salary'].str.contains('year'), 'salary_period'] = 'year'
df_job_data.ix[df_job_data['salary'].str.contains('month'), 'salary_period'] = 'month'
df_job_data.ix[df_job_data['salary'].str.contains('week'), 'salary_period'] = 'week'
df_job_data.ix[df_job_data['salary'].str.contains('day'), 'salary_period'] = 'day'
df_job_data.ix[df_job_data['salary'].str.contains('hour'), 'salary_period'] = 'hour'
df_job_data.ix[df_job_data['salary'].str.contains('p.h'), 'salary_period'] = 'hour'
df_job_data.ix[df_job_data['salary'].str.contains('/hr'), 'salary_period'] = 'hour'
df_job_data.ix[df_job_data['salary'].str.contains('p.d'), 'salary_period'] = 'day'
df_job_data.ix[df_job_data['salary'].str.contains('p.a'), 'salary_period'] = 'year'
df_job_data.ix[df_job_data['salary'].str.contains('pd'), 'salary_period'] = 'day'

In [14]:
#creating and new data frame to work on data that has salaries to clean then and drop all the values form the original df that have slaries. 
#After working on salaries we can concat these two data frames
salary_data_df = df_job_data[df_job_data['salary'] != 'none']
df_job_data = df_job_data[~df_job_data.isin(salary_data_df)].dropna(how='all')
df_job_data['salary'].replace('none',np.nan, inplace=True)
df_job_data['salary'] = df_job_data['salary'].astype('float')

In [15]:
salary_data_df.reset_index(drop=True,inplace=True)

In [16]:
salary_data_df.shape

(717, 6)

In [17]:
#convert the salary having 'to' e.g 80000 to 90000 by '-' e.g 80000 - 90000
#convert 'k' with '000' e.g 80k - 90k will become 80000 - 90000

salary_data_df['salary'] = salary_data_df['salary'].str.replace('to', '-', regex=True)
salary_data_df['salary'] = salary_data_df['salary'].str.replace('k', '000', regex=True)


In [18]:
df_job_data.shape

(1449, 6)

In [19]:
#function to remove all charactres from salary values so we will only be working with numeirc values
def extract_values_with_only_letter(value):
    result = re.sub(r"[a-z]", "", value, flags=re.I)
    final_result = re.sub(r"\W", "", result, flags=re.I)
    return final_result
    

In [23]:
salary_data_df.shape

(717, 6)

In [24]:
#when we put a salary period in data frame based on some character values like p.a, year, month week(e.g 80000-90000 a year).
#There are some yearly salaries which are not able tp generate salary period as year because there is no chracter
#(e.g 8000-90000)
# Therefore, creating a new column called is salary so we can use this column to put the values of salary period as year
#where no period is mentioned.
salary_data_df['is_salary'] = salary_data_df['salary'].apply(lambda x:extract_values_with_only_letter(x))

In [25]:
#convert all the salaries which doesn't have salary value to null and concat to the main data frame
#after calling above function all the slary values that don't have numeric values in them will become empty string.
#so we can drop them from salary_data_df and concat it to df_job_data(all data without salaries)
salary_data_without_salary = salary_data_df[salary_data_df['is_salary']=='']

In [26]:
salary_data_without_salary.shape

(146, 7)

In [27]:
#concating no salary data drom salary_data_df to df_job-data
salary_data_without_salary['salary'] = np.nan
salary_data_without_salary['salary'] = salary_data_without_salary['salary'].astype('float')
df_job_data = pd.concat([df_job_data,salary_data_without_salary])

In [28]:

#drop all rows that only contains characters in salary(no salary data) (is_salary is '')
salary_data_df.drop(salary_data_df[salary_data_df['is_salary']==''].index,inplace=True)


In [29]:
# now after dropping all the rows that don't have any salary information you need to fill remaining salary period column values
#with 'Year' becase the data which is not labled are all of year(e.g 80000-90000, not 80000-90000 a year) 
salary_data_df['salary_period'].fillna('year',inplace=True)

In [30]:
#remove the is_salary column as we don't need
salary_data_df.drop('is_salary',axis=1,inplace=True)

In [31]:
salary_data_df['salary_period'].value_counts()

year     362
day      151
hour      49
week       5
month      4
Name: salary_period, dtype: int64

In [32]:
#generate seprate data frame for each salary period so that the computation will be easy
year_salaries = salary_data_df[salary_data_df['salary_period'] == 'year']
month_salaries = salary_data_df[salary_data_df['salary_period'] == 'month']
week_salaries = salary_data_df[salary_data_df['salary_period'] == 'week']
day_salaries = salary_data_df[salary_data_df['salary_period'] == 'day']
hour_salaries = salary_data_df[salary_data_df['salary_period'] == 'hour']

In [33]:
#all month salary values are not numeric salaries(contains alpha numeric values so can't handeled by earlier regex function)
#so we can make them null and concate it with actual job data
month_salaries['salary'] = np.nan
month_salaries['salary'] = month_salaries['salary'].astype('float')
df_job_data = pd.concat([df_job_data,month_salaries])

In [34]:
df_job_data.shape

(1599, 7)

In [35]:
#Some of the week salary data also have alpha numeric values but not salaries so remove them and concat them to original data frame
df_job_data = pd.concat([df_job_data,week_salaries.iloc[0:3,:]])
df_job_data['salary'] = np.nan
df_job_data['salary'] = df_job_data['salary'].astype('float')

In [36]:
#remove non numeric salary info
week_salaries.drop(week_salaries.iloc[0:3,:]['salary'].index,inplace=True)

In [37]:
df_job_data.shape

(1602, 7)

In [38]:
year_salaries.reset_index(drop=True,inplace=True)

In [39]:
#year salary has some values which are not salaries just text. These are also alphanumeric but not salaries, So dropping them.
df_job_data = pd.concat([df_job_data,year_salaries.iloc[[16,25,41,44,47,48,60,65,67,76,77,86,91,95,101,104,106,118,123,137,142,145,155,170,184,189,197,203,205,215,224,232,234,243,259,263,264,297,306,313,316,62,154,170],:]])
df_job_data['salary'] = np.nan
df_job_data['salary'] = df_job_data['salary'].astype(float)
year_salaries.drop(year_salaries.iloc[[16,25,41,44,47,48,60,65,67,76,77,86,91,95,101,104,106,118,123,137,142,145,155,170,184,189,197,203,205,215,224,232,234,243,259,263,264,297,306,313,316,62,154,170],:].index,inplace=True)


In [40]:
#funciton to extract he numbers from salary information to convert them to numeric salary information
def extract_numbers(value): 
    result = re.sub(r"[a-z]", "", value, flags=re.I)
    final_result = re.sub("[+|()|/]", "", result)
    return final_result

In [41]:
#applying above function to salary of each salary period
hour_salaries['salary'] = hour_salaries['salary'].apply(lambda x: extract_numbers(x))
week_salaries['salary'] = week_salaries['salary'].apply(lambda x: extract_numbers(x))
day_salaries['salary'] = day_salaries['salary'].apply(lambda x: extract_numbers(x))
year_salaries['salary'] = year_salaries['salary'].apply(lambda x: extract_numbers(x))

In [42]:
#Defining a function to handle all possible salary cleaning condition
def split_sal(i,salary_period):
    #try:
   # print(i)
    if '-' in i:
        splt = i.split('-',1)
        #print(i)
        splt[0] = splt[0].strip()
        splt[1] = splt[1].strip()
        if splt[0] == '' and '..' in splt[1]:
            #print(i)
            #print('here1')
            first = float(splt[1].split(' ',1)[0])
            second =float(splt[1].split(' ',1)[0])
            return (first+second)/2
        elif splt[0] != ' ' and '!' not in splt[0] and '..' in splt[1] and '%' not in splt[1] and len(splt[0])>=4:
            #print(i)
            #print('here2')
            first = float(splt[0])
            second = float(splt[1].split(' ',1)[0])
            return (first+second)/2
        elif splt[0] != ' ' and '!' not in splt[0] and '..' in splt[1] and '%' not in splt[1] and len(splt[0])<=3 and salary_period=='year':
            #print(i)
            #print('here3')
            first = float(splt[0]+'000')
            second = float(splt[1].split(' ',1)[0]+'000')
            return (first+second)/2
        elif splt[0] != ' ' and '!' not in splt[0] and '..' in splt[1] and '%' not in splt[1]:
            #print(i)
            #print('here3')
            first = float(splt[0])
            #second = float(splt[1].replace('..',''))
            second = float(splt[1].split(' ',1)[0])
            return (first+second)/2
        elif splt[0] != ' ' and ':' not in splt[0] and  '%' in splt[1] and '..' not in splt[1] and len(splt[1])>3:
            #print(i)
            #print('here4')
            first = float(splt[0].strip())
            second = float(splt[1].split(' ',1)[0])
            return (first+second)/2
        elif splt[0] != ' ' and ':' not in splt[0] and  '%' in splt[1] and '..' not in splt[1] and len(splt[1])<=3: #changed
            #print(i)
            #print('here5')
#             first = float(splt[0])#.strip())
#             second = float(splt[1].split(' ',1)[0])
            return float(splt[0])
        elif splt[0] != ' ' and ':' in splt[0] and  '%' in splt[1] and '..' not in splt[1]:
            #print('here6')
            first = float(splt[0].split(':',1)[1].strip())
            second = float(splt[1].split('  ',1)[0])
            return (first+second)/2
        elif splt[0] != ' ' and '%' in splt[1] and '..' in splt[1]:
            #print('here7')
            first = float(splt[0].strip())
            second = float(splt[1].replace('..','').split('  ',1)[0])
            return (first+second)/2
        elif splt[0] != ' ' and '..' not in splt[0] and '..' not in splt[1] and len(splt[0])>=4 and len(splt[1])>=4 and ' ' not in splt[1] and salary_period == 'year':
            #print(i)
            #print('here8')
            if '.' in splt[0] and '.' in splt[1]:
                first = float(splt[0].split('.',1)[0] + '000')
                second = float(splt[1].split('.',1)[0] + '000')
            else:
                first = float(splt[0])
                second = float(splt[1].strip().split('  ',1)[0])
            return (first+second)/2
        elif splt[0] != ' ' and '..' not in splt[1] and len(splt[0])>=4 and len(splt[1])>=4 and ' ' not in splt[1]:
            #print(i)
            #print('here9')
            first = float(splt[0].split(' ',1)[0])
            second = float(splt[1])
            return (first+second)/2
        elif splt[0] != ' ' and '..' not in splt[1] and len(splt[0])>=4 and len(splt[1])>=4 and ' ' in splt[1] and ' ' not in splt[0]:
            #print('here10')
            first = float(splt[0].strip())
            second = float(splt[1].strip().split(' ',1)[0])
            return (first+second)/2 
        elif splt[0] != ' ' and '..' not in splt[1] and len(splt[0])>=4 and len(splt[1])>=4 and ' ' in splt[1].strip() and ' ' in splt[0].strip():
            #print('here11')
            first = float(''.join(splt[0].strip().split(' ',1)))
            second = float(''.join(splt[1].strip().split(' ',1)))
            return (first+second)/2
        elif splt[0] != '' and len(splt[0])<=3 and len(splt[1])<=3 and salary_period == 'year':#for day
            #print('here16')
            #print(splt[0])
            first = float(splt[0]+'000')
            second = float(splt[1] + '000')
            return (first + second)/2
        elif splt[0] != '' and ':' in splt[0] and len(splt[1])<=3 and salary_period == 'year':#for day
            #print('here16')
            #print(splt[0])
            first = float(splt[0].split(' ',1)[1] +'000')
            second = float(splt[1] + '000')
            return (first + second)/2
        elif splt[0] != '' and '..' not in splt[1] and len(splt[0])<=4 and len(splt[1])<=3 and salary_period == 'day':#for day
            #print('here16')
            #print(splt[0])
            if '.' in splt[0]:
                #print('here')
                first = float(splt[0].replace('.',''))
            else:
                first = float(splt[0])
            second = float(splt[1])
            return (first + second)/2
#         elif splt[0] != '' and '..' in splt[1] and salary_period == 'day':
#             print('here')
#             first = float(splt[0])
#             second = float(splt[1].split(' ',1)[0])
#             return(first+second)/2
        elif splt[0] == '' and '..' not in splt[1] and len(splt[0])<=3 and len(splt[1])<=3 and salary_period == 'day':#for day
            #print('here17')
            return float(splt[1])
        elif splt[0] != '' and '.' not in splt[1] and len(splt[0])<=3 and salary_period == 'hour':
            #print('here 18')
            first = float(splt[0])
            second = float(splt[1])
            return (first + second)/2
        elif splt[0] != '' and '.' in splt[1] and len(splt[0])<=3 and salary_period == 'hour':
            #print('here 19')
            first = float(splt[0])
            second = float(splt[1].split(' ',1)[0])
            return (first + second)/2 
        elif splt[0] != '' and '.' in splt[1] and len(splt[0])>=3 and salary_period == 'hour':
            #print('here 19')
            first = float(splt[0].split('.',1)[0])
            second = float(splt[1].split('.',1)[0])
            return (first + second)/2 
        
        elif splt[0] != ' ' and '!' in splt[0]:
            first = float(splt[0].split(' ',1)[1].strip())
            second = float(splt[1].strip().split(' ',1)[0])
            return (first+second)/2
        elif splt[0]!='' and len(splt[1].strip())>=4 and len(splt[1].strip()) <=6 and len(splt[0].strip())<=3:
            #print('here12')
            if '.' in splt[1] and ' ' not in splt[1] and salary_period == 'year': #changed in year to match with day
                return float(splt[1].split(' ',1)[0] + '000')
            if '.' in splt[1] and salary_period == 'day':
                return float(splt[1].split(' ',1)[0])
            if ' ' in splt[1]:
                return float(splt[1].split(' ',1)[0] + '000')
            else:
                return float(splt[1])#.strip())
        elif splt[0].strip()!='' and len(splt[1].strip()) >=6 and len(splt[0].strip())<=3:
            #print(i)
            #print('here13')
            return float(splt[1].split(' ',1)[0])
        elif splt[0] == '' and '-' in splt[1]:
            #print(i)
            #print('here14')
            value = splt[1].split('-',1)
            first = float(value[0].split('.',1)[1].strip())
            second = float((''.join(value[1].split('.',1))).strip())
            return (first+second)/2
        elif splt[0] == '' and len(splt[1])<=6:
            #print('here15')
            return float(splt[1].strip())
        elif splt[0] == '' and len(splt[1])>6:
            #print('here16')
            return float(splt[1].split(' ',1)[0])
           
    #except:
    else:
        i = i.strip()
        if salary_period == 'year' and len(i.strip())<=3:
            #print('here17')
            if len(i.strip())<=2:
                return float(i.strip()+'000')
            elif len(i.strip())==3:
                return float(i.strip()+'000')
        elif ' ' in i:
            #print('here18')
            return float(i.split(' ',1)[0].strip())
        else: 
            #print('here19')
            return float(i)


In [43]:
#applying this function to all salary information of different salary period
year_salaries['salary']= year_salaries['salary'].apply(lambda x: split_sal(x,'year'))

In [44]:
week_salaries['salary']=week_salaries['salary'].apply(lambda x: split_sal(x,'week'))
week_salaries['salary'] = week_salaries['salary'] * 52

In [45]:
day_salaries['salary'] = day_salaries['salary'].apply(lambda x: split_sal(x,'day'))
day_salaries['salary'] = day_salaries['salary'] * 260

In [46]:
hour_salaries['salary'] = hour_salaries['salary'].apply(lambda x: split_sal(x,'hour'))
hour_salaries['salary'] = hour_salaries['salary'] * 2080

In [47]:
#concatinatinng salary information of all period
all_period_salaries = pd.concat([year_salaries, month_salaries, week_salaries, day_salaries, hour_salaries], axis=0)

In [48]:
all_period_salaries.shape

(525, 6)

In [49]:
df_job_data.shape

(1646, 7)

In [50]:
#concatinating cleaned salaries to original data frame
df_job_data = pd.concat([df_job_data, all_period_salaries], axis=0)

In [51]:
df_job_data.shape

(2171, 7)

In [52]:
#drop all the duplicate job information
df_job_data.drop_duplicates(['company','job_title','location'],inplace=True)

In [53]:
df_job_data.shape

(1856, 7)

In [54]:
df_job_data['location'].value_counts()

Sydney                                    699
Melbourne                                 435
Brisbane                                  206
ACT                                       104
Melbourne VIC                              88
Perth                                      85
Adelaide                                   49
Sydney NSW                                 35
Newcastle, Maitland & Hunter               12
Gosford & Central Coast                    11
Brisbane QLD                               11
Australia                                  11
Gold Coast                                 10
Canberra ACT                                8
Melbourne City Centre VIC                   8
South West Coast VIC                        8
Darwin                                      5
Ballarat & Central Highlands                3
Mulgrave VIC                                3
Wollongong, Illawarra & South Coast         3
Hobart                                      3
Sunshine Coast                    

In [55]:
#function to clean location information
def clean_location(location):
    if 'VIC' in location:
        return 'Melbourne'
    if 'NSW' in location:
        return 'Sydney'
    if 'Brisbane' in location:
        return 'Brisbane'
    if 'Adelaide' in location:
        return 'Adelaide'
    if 'Perth' in location:
        return 'Perth'
    if 'Gold Coast' in location:
        return 'Gold Coast'
    else: 
        return location

In [56]:
df_job_data['location'] = df_job_data['location'].apply(lambda x:clean_location(x))

In [57]:
df_job_data.drop(['is_salary','salary_period'],axis=1,inplace=True)

In [251]:
#saving it to csv(Don't run)
#df_job_data.to_csv('./data/cleaned_job_data')